### NWS Forecast Data

This notebook explains how to scrape, transform, and upload the NWS's hourly weather forecast for the next 6 days** into a dataset in BigQuery. The resulting script can be set to run at regular intervals as an Airflow DAG or a function in Cloud Functions. 

** *For some reason, the hourly forecast doesn't quite extend to a full week, but only 6.5 days. To keep the math easier, we will only scrape the next 6 days -- in the end, this won't affect our pipeline once we have it updating continuously.*

We will collect hourly forecasts for the locations of 23 USCRN data collection stations in Alaska -- the presence of these stations will enable ourselves and any other users of our dataset to evaluate the accuracy of the forecasts.

---

#### Why use scraping over `api.weather.gov`?

Generally speaking, if a website offers an API to access its data then it's a good bet to use it. So why not just use `api.weather.gov`?

The main reason I've chose webscraping for the NWS part of the project is that at times the `api.weather.gov` has given me `500: Internal Server Error` responses even when the HTML data interface is still accessible. The level of information provided is essentially the same:

In [4]:
import pandas as pd
import numpy as np
import requests
import re
import datetime as dt 
import itertools
from bs4 import BeautifulSoup

In [5]:
locations_df = pd.read_csv("../data/locations.csv")

In [10]:
random_location = locations_df.sample(1).iloc[0] 

print(f"{random_location}\n")

lat, lon = random_location['latitude'], random_location['longitude']  

## API results
url = f"https://api.weather.gov/points/{lat},{lon}"

response = requests.get(url)
main_data = response.json()

response = requests.get(main_data['properties']['forecastHourly'])
hourly_data = response.json()
fields = hourly_data['properties']['periods'][0]

print(f"{fields}\n") 

## Webscraping results 
url = f"https://forecast.weather.gov/MapClick.php?lat={lat}&lon={lon}&unit=0&lg=english&FcstType=digital&menu=1"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
df = pd.read_html(str(soup.find_all("table")[5]))[0]
df = df.iloc[1:17,:]

display(df)

station_location    Deadhorse
wbanno                  26565
longitude             -148.46
latitude                70.16
Name: 13, dtype: object

{'number': 1, 'name': '', 'startTime': '2023-02-28T05:00:00-09:00', 'endTime': '2023-02-28T06:00:00-09:00', 'isDaytime': False, 'temperature': -24, 'temperatureUnit': 'F', 'temperatureTrend': None, 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 3}, 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': -33.333333333333336}, 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 79}, 'windSpeed': '5 mph', 'windDirection': 'SW', 'icon': 'https://api.weather.gov/icons/land/night/sct,3?size=small', 'shortForecast': 'Partly Cloudy', 'detailedForecast': ''}



,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
1,Date,02/28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,03/01,NaN,NaN,NaN,NaN
2,Hour (AKST),05,06,07,08,09,10,11,12,13,...,19,20,21,22,23,00,01,02,03,04
3,Temperature (°F),-24,-24,-25,-25,-24,-22,-19,-17,-16,...,-16,-17,-18,-20,-21,-22,-22,-21,-21,-21
4,Dewpoint (°F),-28,-28,-29,-30,-30,-28,-25,-22,-21,...,-20,-23,-22,-24,-24,-25,-25,-25,-26,-26
5,Wind Chill (°F),-38,-41,-41,-41,-44,-42,-39,-41,-39,...,-45,-46,-47,-49,-51,-51,-51,-50,-48,-48
6,Surface Wind (mph),5,6,6,6,9,9,9,15,15,...,22,22,22,22,22,21,21,21,18,18
7,Wind Dir,SW,S,S,S,E,E,E,E,E,...,E,E,E,E,E,E,E,E,E,E
8,Gust,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Sky Cover (%),28,22,22,22,14,14,14,5,5,...,32,32,28,28,28,65,65,65,73,73
10,Precipitation Potential (%),3,3,3,3,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


The results need cleaning (e.g. the column names are in the first row), but you can see that the information is roughly equivalent, with the API offering an `isDaytime` field and the HTML interface offering an actual percentage for sky cover (rather than a snippet like "Partly Cloudy"). Depending on how `isDaytime` is measured, it might be more accurate than any estimate of daylight hours we could make based off the scraped data...

![alaska-sun](../img/alaska_suntimes.png)

Welp -- we'll exclude it for now. Later on we can either access it separately from the API or estimate it ourselves based off standard time tables.

---

#### 1.) Scraping the Data 

For each location, the forecast for the next 48 hours is stored in a tabular data table like this: 

<img src="../img/nws_p1.png" height=400px>

The rest of the forecast can be accessed by jumping ahead in 48 hour increments. We do this by adding `&AheadHour=` on the end of the URL and specifying how many hours (48 and 96). 

The series of transformations required here is quite complex -- a downside of scraping vs using the API -- so I've broken it up into many modular functions.

In [6]:
## General Utilities 
def get_soup(url:str) -> BeautifulSoup:
  """Simple wrapper for getting beautiful soup object from url"""
  result = requests.get(url)
  return BeautifulSoup(result.content, "html.parser") 

def flatten(ls:list): 
  """Flattens/unnests a list of lists by one layer"""
  return list(itertools.chain.from_iterable(ls)) 

def ff_list(ls:list) -> list:
  """Forward fill the values in a list"""
  for i in range(len(ls)):
    if not ls[i] and i > 0:
        ls[i] = ls[i-1]
  return ls

## Specific Utilities
def get_nws_url(row:pd.Series) -> str:
  """
  Get url for the next 48 hours of forecasts from latitude and longitude columns
  
  Args: 
  row (pd.Series): The current row of the dataframe

  Returns: 
  url (str): The url for the next 48 hours of forecasts
  """
  lat, lon = row["latitude"], row["longitude"]
  url = f"https://forecast.weather.gov/MapClick.php?w0=t&w1=td&w2=wc&w3=sfcwind&w3u=1&w4=sky&w5=pop&w6=rh&w7=rain&w8=thunder&w9=snow&w10=fzg&w11=sleet&w12=fog&AheadHour=0&Submit=Submit&FcstType=digital&textField1={lat}&textField2={lon}&site=all&unit=0&dd=&bw=&menu=1"
  return url

def get_last_update(soup:BeautifulSoup) -> dt.datetime:
  """
  Find the "Last Updated" value from a BeautifulSoup object, transform to a datetime in AKST

  Args:
  soup (BeautifulSoup): A Beautiful Soup representation of a particular NWS forecast page

  Returns: 
  last_update_dt (datetime): Datetime representation of time page was last updated (AKST)
  """
  last_update_tag = soup.find('td', string=lambda text: text and 'Last Update:' in text)
  last_update_text = re.sub("Last Update: |\s(?=pm|am)|AKST |,", "", last_update_tag.getText())
  last_update_dt = dt.datetime.strptime(last_update_text, "%I:%M%p %b %d %Y")
  return last_update_dt

In [7]:
## Core helper functions
def extract_table_data(soup:BeautifulSoup, location:str) -> list:
  """
  Extracts 48hr forecast table data from a Beautiful Soup object as a list of lists

  Args: 
  table_records (list): List of <tr> elements containing NWS forecast data

  location (str): The name of the place the forecast is for; used for filling out added "location" column 

  Returns:
  table (list): List of lists containing table data 
  """
  table_records = soup.find_all("table")[5].find_all("tr")

  colspan = table_records[0] # 48hr data is divided into two tables by two colspan elements
  table = [tr for  tr in table_records if tr != colspan] # vertically concat tables by removing colspan elements

  table = [[ele.getText() for ele in tr.find_all("font")] for tr in table] 

  # Add location column 
  location_col = ['location']
  location_col.extend([location]*24) # fill out to match length of other columns
  table.insert(1, location_col)  # for first half of table
  table.insert(19, location_col) # for second half of table

  # Add last_update_nws column 
  last_update_nws = ["last_update_nws"]
  last_update_nws.extend([get_last_update(soup)] * 24)
  table.insert(1, last_update_nws)
  table.insert(19, last_update_nws) 

  return table

def transpose_as_dict(table:list) -> dict:
  """
  Takes the list of lists generated by extract_table_data() and transposes it (flip orientation) by casting as a dictionary
  
  Args:
  table (list): list of lists of columnar data generated by extract_table_data()

  Returns: 
  data_map (dict): Dictionary representation of table, transposed and ready to be made into a dataframe
  """
  data_map = {}
  for col in table: # Table is still "landscape-oriented"
    if col[0] not in data_map.keys(): # cols from first half of table
      data_map[col[0]] = col[1:]
    else: # cols from second half
      data_map[col[0]].extend(col[1:])

  data_map['Date'] = ff_list(data_map['Date'])

  return data_map

def transform_df(fcast_dict:dict) -> pd.DataFrame: 
  """
  Cast dictionary from transpose_as_dict() to a dataframe and transform

  Args: 
  table (list)
  """
  # Create dataframe
  df = pd.DataFrame(fcast_dict)
  
  # Edit column headers 
  df.columns = [col.lower() for col in df.columns] 
  df.rename(columns=lambda x: re.sub('°|\(|\)', '', x), inplace=True)
  df.rename(columns=lambda x: re.sub('%', 'pct', x), inplace=True)
  df.rename(columns=lambda x: re.sub(' ', '_', x.strip()), inplace=True)
  
  # # Replace missing value indicators with Nan -- Leave Validation for SQL step
  # df.replace({'':np.NaN, '--':np.NaN}, inplace=True)
  
  ## Datetime Transformations
  cur_year = dt.datetime.now().year
  dt_strings = df['date'] + '/' + str(cur_year) + ' ' + df['hour_akst'] + ':00 AKST'
  # Local time (AKST)
  df['lst_datetime'] = pd.to_datetime(dt_strings, format='%m/%d/%Y %H:%M AKST')
  # UTC time
  akst_offset = dt.timedelta(hours=9)
  df['utc_datetime'] = df['lst_datetime'] + akst_offset
  
  ## Reorder columns 
  col_names = ['location', 'utc_datetime', 'lst_datetime'] + list(df.columns[4:-2]) + ["last_update_nws"]
  df = df[col_names]


  return df 

And here's the main function to scrape the data:

In [8]:
def get_forecast_df() -> pd.DataFrame:
  """Get a dataframe of NWS forecast data for the next 6 days from various points in Alaska"""

  nws_urls = locations_df.apply(get_nws_url, axis=1)
  url_map = dict(zip(locations_df['station_location'], nws_urls))

  combined_table = []
  for location, url in url_map.items():
    soup_list = [get_soup(url + f"&AheadHour={hr}") for hr in (0,48,96)]
    table_list = flatten([extract_table_data(soup, location) for soup in soup_list])
    combined_table.extend(table_list)
  
  forecast_dict = transpose_as_dict(combined_table)

  return transform_df(forecast_dict)

In [28]:
df = get_forecast_df()
df

,location,utc_datetime,lst_datetime,temperature_f,dewpoint_f,wind_chill_f,surface_wind_mph,wind_dir,gust,sky_cover_pct,precipitation_potential_pct,relative_humidity_pct,rain,thunder,snow,freezing_rain,sleet,fog,last_update_nws
0,Fairbanks,2023-02-28 19:00:00,2023-02-28 10:00:00,-14,-24,-37,15,E,,76,19,57,--,--,SChc,--,--,--,2023-02-28 04:50:00
1,Fairbanks,2023-02-28 20:00:00,2023-02-28 11:00:00,-9,-19,-31,15,E,,76,19,61,--,--,SChc,--,--,--,2023-02-28 04:50:00
2,Fairbanks,2023-02-28 21:00:00,2023-02-28 12:00:00,-6,-14,-26,14,E,,97,79,65,--,--,Ocnl,--,--,--,2023-02-28 04:50:00
3,Fairbanks,2023-02-28 22:00:00,2023-02-28 13:00:00,-2,-10,-22,14,E,,97,79,69,--,--,Ocnl,--,--,--,2023-02-28 04:50:00
4,Fairbanks,2023-02-28 23:00:00,2023-02-28 14:00:00,0,-7,-18,14,E,,97,79,71,--,--,Ocnl,--,--,--,2023-02-28 04:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,Aleknagik,2023-03-06 14:00:00,2023-03-06 05:00:00,29,26,,14,SE,,90,56,88,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3308,Aleknagik,2023-03-06 15:00:00,2023-03-06 06:00:00,30,26,,14,SE,,90,56,86,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3309,Aleknagik,2023-03-06 16:00:00,2023-03-06 07:00:00,30,25,,14,SE,,90,56,83,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3310,Aleknagik,2023-03-06 17:00:00,2023-03-06 08:00:00,30,24,,14,SE,,90,56,81,--,--,Lkly,--,--,--,2023-02-28 03:58:00


#### 2.) Uploading the Data 

Here's the structure of our ETL pipeline as an Entity Relationship Diagram (ERD): 

![TO-DO_ERD]()

The end goal of our pipeline is for the NWS forecasts to be easily evaluated against the historic data from the USCRN. We want to track how the forecast accuracy improves as the forecast gets closer to the current time and to make this easily accessible for analysis later on. We also want to make it easy for data scientists and ML engineers forecasting from the USCRN data to evaluate the performance of their models against the NWS forecasts, with how long the NWS forecasts were made in advance (`utc_datetime - last_update_nws`) being a key parameter. 

By taking a daily snapshot of the NWS forecast data we keep the data highly denormalized, making this sort of analysis much easier and our pipeline simpler to manage. The downside is that we duplicate a lot of data, especially if a forecast for a given time has not changed at all since the day before.

Based on the example table we just made, `3312 * 365 = 1208880` rows added to our main data table per year. We could reduce this greatly by using a nested history column (e.g. a JSON array) but memory is less of a concern for us than ease of analysis.

In [44]:
from yaml import full_load
from google.cloud import bigquery 
from google.oauth2 import service_account 

# GCP/BigQuery information
with open("../config/gcp-config.yaml", "r") as fp:
  gcp_config = full_load(fp)

  
PROJECT_ID = gcp_config['project-id']
DATASET_ID = gcp_config['dataset-id']
STAGING_TABLE_ID = 'nws_staging'
MAIN_TABLE_ID = 'nws' 

# Set credentials
key_path = gcp_config['credentials']
credentials = service_account.Credentials.from_service_account_file(
  key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Create client
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

def load_staging_table(df:pd.DataFrame) -> None: 
  """Upload dataframe from get_forecast_df() to BigQuery staging table"""

  # Set Schema
  schema = [
    bigquery.SchemaField("location", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("utc_datetime", "DATETIME", mode="REQUIRED"), 
    bigquery.SchemaField("lst_datetime", "DATETIME", mode="REQUIRED"), 
    bigquery.SchemaField("temperature_f", "INTEGER", mode="REQUIRED"), 
    bigquery.SchemaField("dewpoint_f", "INTEGER", mode="REQUIRED"), 
    bigquery.SchemaField("wind_chill_f", "INTEGER", mode="NULLABLE"), 
    bigquery.SchemaField("surface_wind_mph", "INTEGER", mode="REQUIRED"), 
    bigquery.SchemaField("wind_dir", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("gust", "INTEGER", mode="NULLABLE"), 
    bigquery.SchemaField("sky_cover_pct", "INTEGER", mode="REQUIRED"), 
    bigquery.SchemaField("precipitation_potential_pct", "FLOAT", mode="REQUIRED"), 
    bigquery.SchemaField("relative_humidity_pct", "FLOAT", mode="REQUIRED"),
    bigquery.SchemaField("rain", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("thunder", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("snow", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("freezing_rain", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("sleet", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("fog", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("last_update_nws", "DATETIME", mode="REQUIRED")
  ] 

  jc = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=False,
    schema=schema,
    create_disposition="CREATE_IF_NEEDED",
    write_disposition="WRITE_TRUNCATE"   
  )

  # Set target table in BigQuery
  full_table_id = f"{PROJECT_ID}.{DATASET_ID}.{STAGING_TABLE_ID}"

  # Upload to BigQuery
  ## If any columns are missing values, include name of column in error message
  try: 
    job = client.load_table_from_dataframe(df, full_table_id, job_config=jc)
    job.result()
  except Exception as e:
    error_message = str(e)
    if 'Required column value for column index' in error_message:
      start_index = error_message.index('Required column value for column index') + len('Required column value for column index: ')
      end_index = error_message.index(' is missing', start_index)
      missing_column_index = int(error_message[start_index:end_index])
      missing_column_name = list(df.columns)[missing_column_index]
      error_message = error_message[:start_index] + f'{missing_column_name} ({missing_column_index})' + error_message[end_index:]
    raise Exception(error_message) 
  
  job = client.load_table_from_dataframe(df, full_table_id, job_config=jc)
  job.result()

  # Log result 
  table = client.get_table(full_table_id)
  print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns into {full_table_id}")

In [45]:
load_staging_table(df)

Loaded 3311 rows and 19 columns into alaska-scrape.weather.nws_staging


Next we make some basic data validations:

In [ ]:

def nan_check() -> None: 
  """Validates data in staging table by checking for NaNs"""

  # The source data uses missing-value indicators so there should be no NaNs
  nan_query = f"""
    SELECT COUNT(*) AS num_rows
    FROM {DATASET_ID}.{STAGING_TABLE_ID}
    WHERE 
      temperature_f IS NULL OR
      dewpoint_f IS NULL OR
      wind_chill_f IS NULL OR
      surface_wind_mph IS NULL OR
      wind_dir IS NULL OR
      gust IS NULL OR
      sky_cover_pct IS NULL OR
      precipitation_potential_pct IS NULL OR
      relative_humidity_pct IS NULL OR
      rain IS NULL OR
      thunder IS NULL OR
      snow IS NULL OR
      freezing_rain IS NULL OR
      sleet IS NULL OR
      fog IS NULL OR
      last_update_nws IS NULL
    """
  
  query_job = client.query(nan_query)
  results = query_job.result()

  num_rows = list(results)[0]['num_rows'] # number of rows containing nans
  if num_rows  > 0:
      print(f"Warning: {num_rows} rows with NaN values found in the dataset")
      # Log bad rows 
  else:
      print("No NaN values detected.")  

In [1]:
def dup_check() -> None: 
  """Validates data in staging table by checking for duplicates in location, utc_datetime, and lst_datetime"""

  # It's conceivable that location/utc_datetime have duplicates but not location/lst_datetime, or vice-versa
  # We check for each possibility rather than duplicates across all of locations/utc_datetime/lst_datetime

  dup_query = f"""
  SELECT *
  FROM (
    SELECT location, utc_datetime, COUNT(*) as count
    FROM {DATASET_ID}.{STAGING_TABLE_ID}
    GROUP BY location, utc_datetime
    HAVING count > 1
  ) AS subquery1
  UNION ALL
  SELECT *
  FROM (
    SELECT location, lst_datetime, COUNT(*) as count
    FROM {DATASET_ID}.{STAGING_TABLE_ID}
    GROUP BY location, lst_datetime
    HAVING count > 1
  ) AS subquery2
  """

  query_job = client.query(dup_query)

  results = query_job.result()

  num_rows = list(results)[0]['num_rows'] # number of rows containing duplicates
  if num_rows  > 0:
      print(f"Warning: {num_rows} rows with duplicate values in location, utc_datetime, and lst_datetime")
      # Log bad rows 
      # Drop duplicates
  else:
      print("No duplicate rows detected")  

Having validated the data and removed any bad rows, we can now insert from our staging table into our main table. In so doing we make two minor changes: 
1. We replace empty values (`""`) with (`0`) in `gust`
2. We add a timestamp column indicating the time of the append (useful if we need to undo something).

In [ ]:
def insert_table() -> None: 
  """Insert staging table into the main data table"""

  insert_query = f"""
    INSERT INTO {DATASET_ID}.{MAIN_TABLE_ID}
    SELECT *, TIMESTAMP()
    FROM {DATASET_ID}.{STAGING_TABLE_ID}
  """



In [43]:
df[df['wind_chill_f'] == ""]

,location,utc_datetime,lst_datetime,temperature_f,dewpoint_f,wind_chill_f,surface_wind_mph,wind_dir,gust,sky_cover_pct,precipitation_potential_pct,relative_humidity_pct,rain,thunder,snow,freezing_rain,sleet,fog,last_update_nws
515,St._Paul,2023-03-04 06:00:00,2023-03-03 21:00:00,34,26,,36,S,,79,51,71,Chc,--,Chc,--,--,--,2023-02-28 03:58:00
516,St._Paul,2023-03-04 07:00:00,2023-03-03 22:00:00,34,26,,36,S,,79,51,73,Chc,--,Chc,--,--,--,2023-02-28 03:58:00
517,St._Paul,2023-03-04 08:00:00,2023-03-03 23:00:00,34,27,,36,S,,79,51,75,Chc,--,Chc,--,--,--,2023-02-28 03:58:00
518,St._Paul,2023-03-04 09:00:00,2023-03-04 00:00:00,34,28,,36,S,,79,51,78,Chc,--,Chc,--,--,--,2023-02-28 03:58:00
519,St._Paul,2023-03-04 10:00:00,2023-03-04 01:00:00,34,28,,36,S,,79,51,80,Chc,--,Chc,--,--,--,2023-02-28 03:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,Aleknagik,2023-03-06 14:00:00,2023-03-06 05:00:00,29,26,,14,SE,,90,56,88,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3308,Aleknagik,2023-03-06 15:00:00,2023-03-06 06:00:00,30,26,,14,SE,,90,56,86,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3309,Aleknagik,2023-03-06 16:00:00,2023-03-06 07:00:00,30,25,,14,SE,,90,56,83,--,--,Lkly,--,--,--,2023-02-28 03:58:00
3310,Aleknagik,2023-03-06 17:00:00,2023-03-06 08:00:00,30,24,,14,SE,,90,56,81,--,--,Lkly,--,--,--,2023-02-28 03:58:00
